In [ ]:
from collections import deque
from copy import deepcopy

# ------------------------------
# Object constructor
# ------------------------------
def make_object(primitive_value=None, primitive_function=None, messages=None):
    """
    Constructs a new Self-like object.
    - primitive_value: if not None, this object is a primitive (e.g., number)
    - primitive_function: if not None, this object acts as a primitive function
    - messages: list of message names (for method-like objects, i.e., function bodies)
    Returns: a Python dict representing the Self object,
    with a slot dictionary, parent list, messages, and primitive data/function info.
    """
    return {
        "slots": {},        # Named slots mapping slot names to object references
        "parents": [],      # List of parent object references for inheritance
        "messages": messages or [],  # Message sequence for method-like/self-evaluating objects
        "primitive_value": primitive_value,  # Optional primitive value (int, str, etc.)
        "primitive_function": primitive_function,  # Optional callable function for primitives
    }

# ------------------------------
# Copy (clone) operation
# ------------------------------
def copy_obj(obj):
    """
    Produces a deep copy (clone) of the given object.
    Deep copying ensures all contained slots/parents/etc. are independent in the copy.
    Cloning is core to Self and replaces class instantiation.
    """
    return deepcopy(obj)

# ------------------------------
# Evaluate an object
# ------------------------------
def evaluate(obj):
    """
    Evaluates a Self object according to its kind:
    - For primitive values: returns a new primitive object (immutability).
    - For primitive functions: calls the function with 'self', returns result or self.
    - For message objects (methods): clones itself, sends all messages, returns last result.
    - For ordinary objects: returns self (identity).
    """
    # Primitive value — evaluate gives a new object with the same value (immutable).
    if obj["primitive_value"] is not None:
        return make_object(primitive_value=obj["primitive_value"])
    # Primitive function — evaluate by calling the function, passing the object as self.
    if obj["primitive_function"] is not None:
        result = obj["primitive_function"](obj)
        return result if result is not None else obj
    # Method object (list of messages) — copy itself, send each message, return last result.
    if obj["messages"]:
        obj_copy = copy_obj(obj)
        result = None
        for msg in obj_copy["messages"]:
            result = sendAMessage(obj_copy, msg)
        return result
    # Regular objects (no primitive, messages, or function): return self.
    return obj

# ------------------------------
# Send a message to an object
# ------------------------------
def sendAMessage(obj, message):
    """
    Sends a message (string) to an object:
    - Looks for the slot with that name.
    - Evaluates the object in the slot.
    - If not found locally, breadth-first search in parent objects.
    - If not found anywhere, raises AttributeError.
    """
    # Try the object's own slots first.
    if message in obj["slots"]:
        return evaluate(obj["slots"][message])
    # Search parent objects using breadth-first search for multiple inheritance.
    queue = deque(obj["parents"])
    visited = set()
    while queue:
        parent = queue.popleft()
        # Avoid infinite loops in cyclic parent graphs
        if id(parent) in visited:
            continue
        visited.add(id(parent))
        # If parent has the slot, evaluate and return it
        if message in parent["slots"]:
            return evaluate(parent["slots"][message])
        # Add the parent's parents to the search queue
        queue.extend(parent["parents"])
    # If not found, raise an error
    raise AttributeError(f"Message '{message}' not found")

# ------------------------------
# Send a message with parameter (for method with parameter passing)
# ------------------------------
def sendAMessageWithParameters(obj, message, parameter_obj):
    """
    Sends a message to the object with a parameter.
    - Finds the slot for the message (locally or by inheritance).
    - Clones the method object.
    - Sets the 'parameter' slot to the argument object.
    - Evaluates the cloned method object.
    """
    # Check own slots first.
    if message in obj["slots"]:
        target = copy_obj(obj["slots"][message])
        target["slots"]["parameter"] = parameter_obj
        return evaluate(target)
    # Search parents breadth-first.
    queue = deque(obj["parents"])
    visited = set()
    while queue:
        parent = queue.popleft()
        if id(parent) in visited:
            continue
        visited.add(id(parent))
        if message in parent["slots"]:
            target = copy_obj(parent["slots"][message])
            target["slots"]["parameter"] = parameter_obj
            return evaluate(target)
        queue.extend(parent["parents"])
    raise AttributeError(f"Message '{message}' not found")

# ------------------------------
# Assign a slot to an object
# ------------------------------
def assignSlot(obj, name, value_obj):
    """
    Assigns/modifies a slot on the object.
    - name: slot name
    - value_obj: object assigned to that slot
    """
    obj["slots"][name] = value_obj

# ------------------------------
# Manage parent relationships (object references)
# ------------------------------
def addParent(obj, parent_obj):
    """
    Adds a parent to the object's parent list, avoiding duplicates.
    - This controls inheritance and is separate from slots to allow anonymous parents.
    """
    if parent_obj not in obj["parents"]:
        obj["parents"].append(parent_obj)

def assignParentSlot(obj, name, parent_obj):
    """
    Assigns a parent object both:
    - as a named slot (so it can be found by name),
    - and as an entry in the parent's list for inheritance traversal.
    """
    assignSlot(obj, name, parent_obj)
    addParent(obj, parent_obj)

# ------------------------------
# Human-readable print (tree representation, handles cycles)
# ------------------------------
def print_object(obj, tab=0, seen=None):
    """
    Recursively prints an easily readable tree representation of the object.
    - Shows all slots, parent slots (marked), and handles cycles to prevent infinite recursion.
    - tab: current indentation level (for readability)
    - seen: set of visited object ids (to detect and label cycles)
    """
    if seen is None:
        seen = set()
    indent = "  " * tab
    if id(obj) in seen:
        return indent + "<circular>"  # Detect circular references
    seen.add(id(obj))
    parts = []
    # Label object type line: value, function, method/messages, or plain object
    if obj["primitive_value"] is not None:
        parts.append(f"{indent}<PrimitiveValue {obj['primitive_value']}>")
    elif obj["primitive_function"] is not None:
        parts.append(f"{indent}<PrimitiveFunction>")
    elif obj["messages"]:
        parts.append(f"{indent}<MethodObject messages={obj['messages']}>")
    else:
        parts.append(f"{indent}<Object>")
    # Show slots and recursively print their contents
    for slot, value in obj["slots"].items():
        parent_tag = " [parent]" if value in obj["parents"] else ""
        parts.append(f"{indent}  {slot}:{parent_tag}")
        parts.append(print_object(value, tab+2, seen))
    # Show any parents not already listed as slots ("anonymous" parents)
    non_slot_parents = [p for p in obj["parents"] if p not in obj["slots"].values()]
    if non_slot_parents:
        for pi, p in enumerate(non_slot_parents):
            parts.append(f"{indent}  <anonymous parent #{pi}> [parent]:")
            parts.append(print_object(p, tab+2, seen))
    return "\n".join(parts)


In [39]:
# ---- EXAMPLE USAGE FOR EACH FUNCTION ----

# 1. make_object: Create new Self-like objects
number_obj = make_object(primitive_value=42)
hello_obj = make_object(primitive_value="Hello, world!")
func_obj = make_object(primitive_function=lambda self: print("Primitive function called!"))
method_obj = make_object(messages=['say_hello'])  # Will expect 'say_hello' slot when evaluated

# 2. assignSlot: Assign/mutate slots on an object
assignSlot(number_obj, "description", hello_obj)
# number_obj now has a slot "description" referencing hello_obj

# 3. addParent: Make a parent object for inheritance traversal
another_obj = make_object(primitive_value="I am your parent!")
addParent(number_obj, another_obj)
# number_obj parents now include another_obj

# 4. assignParentSlot: Add "special" parent as both slot and inheritance parent
special_parent = make_object(primitive_value="Special parent here!")
assignParentSlot(number_obj, "special", special_parent)
# number_obj has both a slot "special" and inheritance to special_parent

# 5. copy_obj: Deeply clone an object (prototype inheritance)
number_obj_clone = copy_obj(number_obj)
print("Primitive value of clone:", number_obj_clone["primitive_value"])

# 6. evaluate: Evaluate each type of object
print("Evaluate primitive value:", evaluate(number_obj)["primitive_value"])    # Should print 42
print("Evaluate primitive function:") 
evaluate(func_obj)  # Should print: Primitive function called!
# For message objects, first assign the 'say_hello' slot
assignSlot(method_obj, "say_hello", func_obj)
evaluate(method_obj)  # Should trigger the primitive function and print

# 7. sendAMessage: Send a message (slot name) to an object
assignSlot(another_obj, "greeting", hello_obj)
print("Send message 'greeting':", sendAMessage(another_obj, "greeting")["primitive_value"])  # Should be "Hello, world!"
# Test inheritance
print("Send message 'special':", sendAMessage(number_obj, "special")["primitive_value"])     # Should be "Special parent here!"

# 8. sendAMessageWithParameters: Calls a method-like object with a parameter
def echo_param(obj):
    param = obj["slots"].get("parameter")
    if param and "primitive_value" in param:
        print("Got parameter:", param["primitive_value"])
    return obj

echo_obj = make_object(primitive_function=echo_param)
assignSlot(another_obj, "echo", echo_obj)
param_example = make_object(primitive_value="ParamValue")
sendAMessageWithParameters(another_obj, "echo", param_example)  # Should print parameter value

# 9. print_object: Pretty print the object and its slots/parents (handles cycles)
print("\nObject tree for number_obj:")
print(print_object(number_obj))

print("\nObject tree for method_obj:")
print(print_object(method_obj))


Primitive value of clone: 42
Evaluate primitive value: 42
Evaluate primitive function:
Primitive function called!
Primitive function called!
Send message 'greeting': Hello, world!
Send message 'special': Special parent here!
Got parameter: ParamValue

Object tree for number_obj:
<PrimitiveValue 42>
  description:
    <PrimitiveValue Hello, world!>
  special: [parent]
    <PrimitiveValue Special parent here!>
  <anonymous parent #0> [parent]:
    <PrimitiveValue I am your parent!>
      greeting:
        <circular>
      echo:
        <PrimitiveFunction>

Object tree for method_obj:
<MethodObject messages=['say_hello']>
  say_hello:
    <PrimitiveFunction>
